<a href="https://colab.research.google.com/github/YonatanEliyahu/FinalProject/blob/main/Final_Project_model_Tranings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drowsiness and distraction dedector 

## import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob as gb
import tensorflow as tf

## import data


### create train df

In [ ]:
size=94
train_dir='/content/drive/MyDrive/FinalProject/data'

train_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.15,
).flow_from_directory(train_dir,batch_size=16,target_size=(size,size),subset="training",shuffle=True)


Found 12033 images belonging to 4 classes.


In [ ]:
classes=list(train_generator.class_indices.keys())
def print_examples(DataFrame,fClasses):
  plt.figure(figsize=(20,20))
  for X_batch, y_batch in train_generator:
      # create a grid of 4x4 images
     for i in range(0,16):
         plt.subplot(4,4,i+1)
         plt.imshow(X_batch[i])
         plt.title(fClasses[np.where(y_batch[i]==1)[0][0]])
     # show the plot
     plt.show()
     break

print_examples(train_generator,classes)

### create validation df

In [ ]:
valid_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.15,
).flow_from_directory(train_dir,batch_size=16,target_size=(size,size),subset='validation',shuffle=True)
#print_examples(valid_generator,classes)

Found 2122 images belonging to 4 classes.


### create test df

In [ ]:
test_dir='/content/drive/MyDrive/FinalProject/test_data'
test_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0
).flow_from_directory(test_dir,batch_size=16,target_size=(size,size),shuffle=False)
#print_examples(test_generator,classes)

Found 1300 images belonging to 4 classes.


In [ ]:
print(len(valid_generator)*16)
print(len(test_generator)*16)
print(classes)

2128
1312
['close_look', 'forward_look', 'left_look', 'right_look']


## Binary Model

The following cells will create a classification model (one vs rest) that will specialize in identification of closed eyes.

V2 model - [link to model](https://drive.google.com/file/d/18gtEx_Xu-exw4drQ08MhPV8Hf4gep99F/view?usp=sharing)

Test loss: 0.0345

Test accuracy: 0.9892

V3 model - [link to model](https://drive.google.com/file/d/1Awy9pPmmoq5UMDgQcpdNOftsk9j_0alw/view?usp=share_link)

Test loss: 0.0058

Test accuracy: 0.9985

In [ ]:
#TensorFlow is a free and open-source software library for machine learning and artificial intelligence.
# It can be used across a range of tasks but has a particular focus on training and inference of deep neural networks.
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization ,GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau ,ModelCheckpoint

In [ ]:
# Define the labels for binary classification
binary_labels = ['close_look', 'open_look']

def create_binary_df(df):
  binary_generator=df
  binary_generator.classes = (df.classes == 0).astype('int')
  binary_generator.class_mode = 'binary'
  binary_generator.class_indices = {label: i for i, label in enumerate(binary_labels)}
  return binary_generator

binary_train_generator = create_binary_df(train_generator)
binary_valid_generator = create_binary_df(valid_generator)
binary_test_generator  = create_binary_df(test_generator)
binary_classes=list(binary_train_generator.class_indices.keys())
print(binary_classes)

['close_look', 'open_look']


In [ ]:
binary_classes=list(binary_test_generator.class_indices.keys())
print(binary_classes)

['close_look', 'open_look']


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

batch_size = 16
# Create a simple binary classification model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(size, size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model on the binary classification generators
history = model.fit(binary_train_generator,
                    steps_per_epoch=binary_train_generator.samples // batch_size ,
                    epochs=8, 
                    validation_data=binary_valid_generator,
                    validation_steps=binary_valid_generator.samples // batch_size )

# Evaluate the model on the test set
loss, accuracy = model.evaluate(binary_test_generator, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Get the model predictions on the test set
y_pred = model.predict(binary_test_generator)
y_pred_classes = (y_pred > 0.5).astype('int')

# Generate a classification report and confusion matrix
print('Classification Report')
print(classification_report(binary_test_generator.classes, y_pred_classes, target_names=binary_labels))
print('Confusion Matrix')
print(confusion_matrix(binary_test_generator.classes, y_pred_classes))

Epoch 1/8
752/752 [==============================] - 4212s 6s/step - loss: 0.1710 - accuracy: 0.9372 - val_loss: 0.0943 - val_accuracy: 0.9763
Epoch 2/8
752/752 [==============================] - 263s 350ms/step - loss: 0.1017 - accuracy: 0.9673 - val_loss: 0.0600 - val_accuracy: 0.9830
Epoch 3/8
752/752 [==============================] - 271s 361ms/step - loss: 0.0661 - accuracy: 0.9786 - val_loss: 0.0428 - val_accuracy: 0.9834
Epoch 4/8
752/752 [==============================] - 266s 353ms/step - loss: 0.0393 - accuracy: 0.9868 - val_loss: 0.0515 - val_accuracy: 0.9863
Epoch 5/8
752/752 [==============================] - 263s 350ms/step - loss: 0.0341 - accuracy: 0.9881 - val_loss: 0.0333 - val_accuracy: 0.9910
Epoch 6/8
752/752 [==============================] - 269s 357ms/step - loss: 0.0272 - accuracy: 0.9910 - val_loss: 0.0339 - val_accuracy: 0.9877
Epoch 7/8
752/752 [==============================] - 266s 353ms/step - loss: 0.0199 - accuracy: 0.9938 - val_loss: 0.0348 - val_accu

In [ ]:
model.save('/content/drive/MyDrive/FinalProject/binary_model_v3.h5')

## Four classes model

Those code cells will create classification model that will classify between the four classes ['close_look', 'forward_look', 'left_look', 'right_look'] 


model_v2 - [link to model](https://drive.google.com/file/d/1-1wNrOXXS3ruL_IUMbXPVl37GBldVm6b/view?usp=sharing)

Test loss: 0.0444

Test accuracy: 0.9908

model_v3 - [link to model](https://drive.google.com/file/d/1--EoGg25h2E8BpmWXv6yebM2n_tZueLI/view?usp=sharing)


Test loss: 0.0678

Test accuracy: 0.9877


To create a new model you should recreate the tarin, validation and test data frames.



In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential

# Define model architecture
model = Sequential([
    Conv2D(32, 3, activation='relu', kernel_initializer='he_normal', input_shape=(size, size, 3)),
    Conv2D(64, 3, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    MaxPooling2D(3),
    Conv2D(128, 3, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    MaxPooling2D(3),
    Conv2D(256, 3, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    MaxPooling2D(3),
    Flatten(),
    Dense(64, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    Dense(4, activation='softmax', kernel_initializer='glorot_normal')
])

# Compile model
optimizer_adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
model.compile(optimizer=optimizer_adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stop = EarlyStopping(patience=5, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)

# Train model with data generators
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=50,
    steps_per_epoch=len(train_generator),
    validation_steps=len(valid_generator),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


Epoch 1/50
753/753 [==============================] - 763s 997ms/step - loss: 0.3606 - accuracy: 0.8752 - val_loss: 0.5399 - val_accuracy: 0.8648 - lr: 0.0010
Epoch 2/50
753/753 [==============================] - 723s 960ms/step - loss: 0.1839 - accuracy: 0.9418 - val_loss: 0.1494 - val_accuracy: 0.9628 - lr: 0.0010
Epoch 3/50
753/753 [==============================] - 738s 980ms/step - loss: 0.1332 - accuracy: 0.9600 - val_loss: 0.2194 - val_accuracy: 0.9194 - lr: 0.0010
Epoch 4/50
753/753 [==============================] - 706s 938ms/step - loss: 0.1082 - accuracy: 0.9673 - val_loss: 1.6072 - val_accuracy: 0.8129 - lr: 0.0010
Epoch 5/50
753/753 [==============================] - ETA: 0s - loss: 0.0890 - accuracy: 0.9721
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
753/753 [==============================] - 738s 979ms/step - loss: 0.0890 - accuracy: 0.9721 - val_loss: 0.3766 - val_accuracy: 0.9237 - lr: 0.0010
Epoch 6/50
753/753 [=======================

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Test loss: 0.0678
Test accuracy: 0.9877


In [ ]:
model.save('/content/drive/MyDrive/FinalProject/fourWayModel_v3.h5')